In [1]:
import sys
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [2]:
from keras.utils import to_categorical 
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.backend.tensorflow_backend as K
from sklearn.model_selection import train_test_split
import numpy as np
import os
import tensorflow as tf
tf.__version__

Using TensorFlow backend.


'1.12.0'

In [3]:
npz_files = np.load('./urban8K_STFT_All-002.npz')

In [4]:
npz_files.files

['X', 'y', 'fn_s']

In [5]:
X = npz_files['X']
y = npz_files['y']
fn_s = npz_files['fn_s']

In [6]:
X.shape, y.shape

((7333, 89010), (7333,))

In [7]:
y[0], fn_s[0]

(8.0, 'Data/UrbanSound8K/audio/fold10/159742-8-0-2.wav')

In [8]:
y = to_categorical(y)

In [9]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [39]:
shuffle_index = np.random.permutation(len(X))
X = X[shuffle_index]
y = y[shuffle_index]
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
X_train.shape

(5866, 89010)

In [41]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_train.shape[1],1)

In [42]:
X_train.shape, X_test.shape

((5866, 89010, 1), (1467, 89010, 1))

In [43]:
y_train.shape, y_test.shape

((5866, 10), (1467, 10))

In [54]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import merge, Convolution2D, MaxPooling1D, Input, add, Convolution1D, Conv1D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ReLU


def makeModel(nbChannels, shape1, shape2, nbClasses, nbRCL=5, nbFilters=128, filtersize = 3):


	model = BuildRCNN(nbChannels, shape1, shape2, nbClasses, nbRCL, nbFilters, filtersize)
	return model

def BuildRCNN(nbChannels, shape1, shape2, nbClasses, nbRCL, nbFilters, filtersize):
    
    def RCL_block(l_settings, l, pool=True, increase_dim=False):
        input_num_filters = l_settings.output_shape[1]
        if increase_dim:
            out_num_filters = input_num_filters*2
        else:
            out_num_filters = input_num_filters
		   
        conv1 = Conv1D(out_num_filters, 1, padding='same')
        stack1 = conv1(l)   	
        stack2 = BatchNormalization()(stack1)
        stack3 = ReLU()(stack2)
        
        conv2 = Conv1D(out_num_filters, filtersize, padding='same', init = 'he_normal')
        stack4 = conv2(stack3)
        stack5 = add([stack1, stack4])
        stack6 = BatchNormalization()(stack5)
        stack7 = ReLU()(stack6)
    	
        conv3 = Conv1D(out_num_filters, filtersize, padding='same', weights = conv2.get_weights())
        stack8 = conv3(stack7)
        stack9 = add([stack1, stack8])
        stack10 = BatchNormalization()(stack9)
        stack11 = ReLU()(stack10)    
        
        conv4 = Conv1D(out_num_filters, filtersize, padding='same', weights = conv2.get_weights())
        stack12 = conv4(stack11)
        stack13 = add([stack1, stack12])
        stack14 = BatchNormalization()(stack13)
        stack15 = ReLU()(stack14)    
        
        if pool:
            stack16 = MaxPooling1D(pool_size = 3, padding='same')(stack15) 
            stack17 = Dropout(0.1)(stack16)
        else:
            stack17 = Dropout(0.1)(stack15)
            
        return stack17

    #Build Network
    input_img = Input(shape=(shape1, shape2))
    conv_l = Conv1D(nbFilters, filtersize, padding='same', activation='relu')
    l = conv_l(input_img)
    
    for n in range(nbRCL):
        if n % 2 ==0:
            l = RCL_block(conv_l, l, pool=False)
        else:
            l = RCL_block(conv_l, l, pool=True)
    
    out = Flatten()(l)        
    l_out = Dense(nbClasses, activation = 'softmax')(out)
    
    model = Model(input = input_img, output = l_out)
    
    return model


In [56]:
model = makeModel(nbChannels=1, shape1=X_train.shape[1], shape2=X_train.shape[2], nbClasses=y_train.shape[1])

/home/user4/.conda/envs/tensor/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(89010, 3, padding="same", kernel_initializer="he_normal")`


ResourceExhaustedError: OOM when allocating tensor with shape[3,89010,89010] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node conv1d_88/truncated_normal/TruncatedNormal (defined at /home/user4/.conda/envs/tensor/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185)  = TruncatedNormal[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=6056278, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1d_88/truncated_normal/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'conv1d_88/truncated_normal/TruncatedNormal', defined at:
  File "/home/user4/.conda/envs/tensor/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/user4/.conda/envs/tensor/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/user4/.conda/envs/tensor/l

In [35]:
epochs = 20
batch_size = 100

pat = 5
early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

MODEL_SAVE_FOLDER_PATH = './model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + 'urban_sound2_functional_RCNN.h5'
model_checkpoint = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, model_checkpoint], verbose=1, validation_split=0.1)  

Train on 5279 samples, validate on 587 samples
Epoch 1/20
5279/5279 [==============================] - 18s 3ms/step - loss: 1.8052 - acc: 0.3550 - val_loss: 1.6701 - val_acc: 0.4191

Epoch 00001: val_loss improved from inf to 1.67005, saving model to ./model/urban_sound2_functional_RCNN.h5
Epoch 2/20
5279/5279 [==============================] - 9s 2ms/step - loss: 1.3708 - acc: 0.5143 - val_loss: 2.0684 - val_acc: 0.3475

Epoch 00002: val_loss did not improve from 1.67005
Epoch 3/20
5279/5279 [==============================] - 8s 2ms/step - loss: 1.1878 - acc: 0.5768 - val_loss: 1.8732 - val_acc: 0.4344

Epoch 00003: val_loss did not improve from 1.67005
Epoch 4/20
5279/5279 [==============================] - 9s 2ms/step - loss: 1.0138 - acc: 0.6450 - val_loss: 1.3137 - val_acc: 0.5724

Epoch 00004: val_loss improved from 1.67005 to 1.31367, saving model to ./model/urban_sound2_functional_RCNN.h5
Epoch 5/20
5279/5279 [==============================] - 8s 2ms/step - loss: 0.8585 - acc: 